此脚本用于Grep命令（grep -aiE "'03046.*2022-09-" *.txt -B1 | grep -aiE "^T.*2022-" > status.csv）之后

第一步将报警信息附加到状态变化信息之后

第二步插值，在状态变化之前3s插入上个变化的信息，如果两个状态变化跨越自然小时，则在整点前后各插入上个状态变化的信息

In [ ]:
import pandas as pd
#import os
#import datetime
#import tkinter.filedialog as tk
#import tkinter

In [ ]:
logpath = r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError\Data\GeneralLogs\status.csv"

In [ ]:
df1 = pd.read_csv(logpath, sep="\t", encoding="utf_8_sig", header=None, parse_dates=[2])
df1.head()

In [ ]:
df1.drop([1,3,4,6,7,8,9],inplace=True,axis=1)
df1.rename(columns={0 : "rowInfo", 2 : "dt", 5 : "statusChange"},inplace=True)
df1.head()

In [ ]:
df1.dropna(inplace=True)

In [ ]:
def sn_extract(strs):
    sn = strs.split(".")[0]
    return sn

In [ ]:
df1.loc[:, "sn"] = df1.rowInfo.map(sn_extract)
df1.loc[:, "code"] = df1.rowInfo.str.slice(-6,)
df1.drop("rowInfo", inplace=True, axis=1)
df1.head()

In [ ]:
df2 = df1.copy()
df2.index += 1

In [ ]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=["","_pre"])
df1 = df1.loc[(df1.sn == df1.sn_pre) & (df1.code == "'03046"), :]
df1.drop(["statusChange_pre", "sn_pre", "code"], inplace=True, axis=1)
df1.reset_index(inplace=True, drop=True)
df1.head()

In [ ]:
df2 = df1.copy()
df2.drop(["statusChange", "dt_pre", "code_pre"], inplace=True, axis=1)
df1.index += 1
df1.head()

In [ ]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True,suffixes=["", "_next"])
df1.head()

In [ ]:
df1 = df1.loc[df1.sn == df1.sn_next, :]
df1.drop("sn_next", inplace=True, axis=True)
df1.head()

In [ ]:
df1.reset_index(inplace=True, drop=True)

In [ ]:
replace_dic = {
    "开机": "Power up",
    "初始化": "Initializing",
    "维护": "Maintenance",
    "错误": "Error",
    "紧急停止": "Emergency stop",
    "忙": "Busy",
    "诊断": "Diagnostics",
    "准备": "Ready",
    "受控停机": "Controlled stop",
    "未连接": "Not connected",
    "温度调整": "Adjusting thermal",
    "分析仪状态从": "Analyzer Status changed from",
    "变为": "to"
}

def replace_desp(desp):
    if "分析仪状态从" not in desp:
        return desp
    for (cn, en) in replace_dic.items():
        desp = desp.replace(cn, en)
    return desp

In [ ]:
df1["statusChange"] = df1["statusChange"].map(replace_desp)
df1.head()

In [ ]:
df1 = df1.loc[:, ["sn", "dt", "statusChange", "dt_next", "dt_pre", "code_pre"]]
df1.head()

In [ ]:
l = []

In [ ]:
def insert_rows(df_row):
    global l
    if df_row["dt_next"] - df_row["dt"] < pd.Timedelta(3,"s"):
        l.append(df_row.to_dict())
    elif df_row.dt_next.strftime("%Y%m%d%H") == df_row["dt"].strftime("%Y%m%d%H"):
        l.append(df_row.to_dict())
        l.append({"sn" : df_row.sn, "dt" : df_row.dt_next - pd.Timedelta(2,"s"), "statusChange" : df_row.statusChange, "dt_next" : df_row.dt_next, "dt_pre" : df_row.dt_pre, "code_pre" : df_row.code_pre})
    else:
        l.append(df_row.to_dict())
        dt_tuple = df_row["dt"].timetuple()
        dt_next_tuple = df_row.dt_next.timetuple()
        dt_hourEnd = pd.Timestamp(dt_tuple.tm_year,dt_tuple.tm_mon,dt_tuple.tm_mday,dt_tuple.tm_hour) + pd.Timedelta('0 days 00:59:59.9')
        dt_next_hourStart = pd.Timestamp(dt_next_tuple.tm_year,dt_next_tuple.tm_mon,dt_next_tuple.tm_mday,dt_next_tuple.tm_hour)
        while dt_hourEnd < dt_next_hourStart:
            l.append({"sn" : df_row.sn, "dt" : dt_hourEnd, "statusChange" : df_row.statusChange, "dt_next" : df_row.dt_next, "dt_pre" : df_row.dt_pre, "code_pre" : df_row.code_pre})
            l.append({"sn" : df_row.sn, "dt" : dt_hourEnd + pd.Timedelta('0 days 00:00:0.11') , "statusChange" : df_row.statusChange, "dt_next" : df_row.dt_next, "dt_pre" : df_row.dt_pre, "code_pre" : df_row.code_pre})
            dt_hourEnd += pd.Timedelta('0 days 01:00:00')
        l.append({"sn" : df_row.sn, "dt" : df_row.dt_next - pd.Timedelta(2,"s"), "statusChange" : df_row.statusChange, "dt_next" : df_row.dt_next, "dt_pre" : df_row.dt_pre, "code_pre" : df_row.code_pre})

In [ ]:
df1.apply(insert_rows,axis=1)

In [ ]:
df1 = pd.DataFrame.from_records(l)
df1

In [ ]:
df1.drop("dt_next", inplace=True, axis=True)
df1.head()

In [ ]:
df2 = df1.copy()
df1.index += 1
df1.head()

In [ ]:
df2.drop(["statusChange", "dt_pre", "code_pre"], inplace=True, axis=True)
df2.head()

In [ ]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=["", "_next"])

In [ ]:
df1.head()

In [ ]:
df1 = df1.loc[df1.sn == df1.sn_next, :]
df1.drop("sn_next", inplace=True, axis=1)
df1.reset_index(drop=True, inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.loc[:,"periodH"] = (df1["dt_next"] - df1["dt"]) / pd.Timedelta(1,"h")
df1.loc[:,"date"] = df1["dt"].dt.strftime("%Y/%m/%d")
df1.loc[:,"weekday"] = df1["dt"].dt.weekday + 1
df1.loc[:,"oClock"] = df1["dt"].dt.hour

In [ ]:
df1.drop("dt_next", inplace=True, axis=True)
df1.head()

In [ ]:
df1.loc[:, "validTimeDiff"] = (df1.dt - df1.dt_pre) / pd.Timedelta(1,"s")
df1.loc[:, "validTimeDiff"] = df1.validTimeDiff.map(lambda x: "y" if x < 2 else "n")

In [ ]:
df1.head()

In [ ]:
def status_extract(strs):
    status = strs.split(" to ")[1]
    return status[0:-1]

In [ ]:
df1.loc[:, "status"] = df1.statusChange.map(status_extract)
df1.head()

In [ ]:
df1.to_csv("status_ecg.csv")